# Tests of reading in excel spreadsheet

This has been exported from google sheets. I am particularly interested in being able to get hold of the notes

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
datapath = Path.cwd().parent.parent / "data" / "spec1d"

In [ ]:
df = pd.read_excel(datapath / "All-Lines-MUSE-NGC-346.xlsx")

In [ ]:
df

So, that gave me the table, but I do not see any of the notes.

In [ ]:
set(str(x).rstrip('?') for x in df.Type)

In [ ]:
import openpyxl

In [ ]:
workbook = openpyxl.load_workbook(datapath / "All-Lines-MUSE-NGC-346.xlsx", data_only=True)
sheet = workbook.active
sheet

In [ ]:
pd.DataFrame(list(sheet.values)[1:], columns=list(sheet.values)[0])


In [ ]:
cell = sheet['A1']

In [ ]:
cell.comment.text, cell.comment.author

Aha, so it turns out that the notes are called "Comments" in the excel file. OK, that is easy then.

In [ ]:
pd.DataFrame(
    [
        [
            # Take content from non-empty comments
            x.comment.content if x.comment else ""
            for x in row
            # Only use the columns we want
            if x.column_letter in "ABCDEFGHI"
        ]
        for row in sheet.rows
        # And only use rows that have at least some data
        if any(x.value for x in row)
    ]
)

In [ ]:
pd.read_excel??

In [ ]:
any(x.value for x in list(sheet.rows)[468])

So, the above works for automatically extracting all the notes in the same shape as the original table

What happens when it is a Sheets "comment" rather than "note"?

In [ ]:
sheet['A468'].comment


The author is included in the content field, while the author field is still `None`

What happens when we have several comments on the same cell?

In [ ]:
x = sheet['F455'].comment
x

In [ ]:
x.content

They all get concatenated into the `.content` field